In [1]:
import pandas as pd
import numpy as np
import time
import requests

In [2]:
cleaning_time_start = time.time()

# Load the data
data = pd.read_csv("../../data/raw/environment data.csv")
# Strip extra spaces from column names
data.rename(columns=lambda x: x.strip(), inplace=True)
new_data = data.copy()

# Insert column "COUNTY" into 4th index with no values and allowing duplicates
new_data.insert(4, "COUNTY", None, True)

In [3]:
# CONVERT LAT AND LONG --> COUNTY NAME
stations = new_data.STATION.unique()
start = time.time()
for station in stations:
    # get lat and long values from the new data for that station
    latitude, longitude = new_data[new_data['STATION'] == station].values[0][2:4]
    # create json payload with corresponding lat long values
    payload = { 'latitude': latitude, 'longitude': longitude, 'format': 'json' }
    r = requests.get('https://geo.fcc.gov/api/census/area', params=payload).json()
    # get county names for each lat long and fill in new_data
    county = r['County']['name']
    new_data.loc[new_data['STATION'] == station, 'COUNTY'] = county
print("County names generated in {} seconds.".format(time.time() - start))

# Write to file
new_data.to_csv("../../data/raw/environment data - with counties.csv", index=False)

County names generated in 17.299970388412476 seconds.


In [4]:
# CLEANING THE DATA FURTHER
# load the data
data = pd.read_csv("../../data/raw/environment data - with counties.csv")
# replace cells with only spaces with NaN
df = data.copy().replace(r'^\s*$', np.nan, regex=True)

# Isolate relevant columns
relevant_cols = ["NAME", "LATITUDE", "LONGITUDE", "COUNTY", "ELEVATION", "DATE", "CDSD", "EMXP", "PRCP", "CLDD", "DT00", "DT32", "DX32", "DX70", "DX90", "EMNT", "EMXT", "FZF0", "FZF1", "FZF2", "FZF3", "FZF4", "FZF5", "FZF6", "FZF7", "FZF8", "FZF9", "HTDD", "TAVG", "TMAX", "TMIN", "SNOW", "PSUN"]

# Isolate data from relevant columns and drop duplicate rows
relevant = df[relevant_cols].drop_duplicates()
# drop columns with no null values / we don't care about for cleaning
relevant = relevant.sort_values(by=["COUNTY", "DATE"]).reset_index(drop=True).drop(["NAME", "LATITUDE", "LONGITUDE", "ELEVATION"], axis=1)
relevant.head(3)

,COUNTY,DATE,CDSD,EMXP,PRCP,CLDD,DT00,DT32,DX32,DX70,...,FZF6,FZF7,FZF8,FZF9,HTDD,TAVG,TMAX,TMIN,SNOW,PSUN
0,Addison,1990,NaN,2.52,54.87,NaN,NaN,NaN,55.0,107.0,...,NaN,NaN,NaN,NaN,NaN,NaN,54.2,NaN,131.3,NaN
1,Addison,1990,NaN,2.60,48.94,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.2,NaN
2,Addison,1990,NaN,2.40,46.65,NaN,2.0,136.0,NaN,NaN,...,24.0,24.0,17.0,11.0,7198.0,NaN,NaN,38.5,69.4,NaN


In [5]:
# group the data by county and date
relevant_groupby = relevant.groupby(["COUNTY", "DATE"])
relevant_groupby.head()

,COUNTY,DATE,CDSD,EMXP,PRCP,CLDD,DT00,DT32,DX32,DX70,...,FZF6,FZF7,FZF8,FZF9,HTDD,TAVG,TMAX,TMIN,SNOW,PSUN
0,Addison,1990,NaN,2.52,54.87,NaN,NaN,NaN,55.0,107.0,...,NaN,NaN,NaN,NaN,NaN,NaN,54.2,NaN,131.3,NaN
1,Addison,1990,NaN,2.60,48.94,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.2,NaN
2,Addison,1990,NaN,2.40,46.65,NaN,2.0,136.0,NaN,NaN,...,24.0,24.0,17.0,11.0,7198.0,NaN,NaN,38.5,69.4,NaN
3,Addison,1991,NaN,1.90,37.15,NaN,30.0,161.0,63.0,109.0,...,27.0,20.0,20.0,11.0,NaN,42.9,53.6,32.3,89.4,NaN
4,Addison,1991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1216,Windsor,2019,NaN,2.17,47.30,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1217,Windsor,2019,NaN,2.37,43.76,NaN,19.0,166.0,NaN,NaN,...,28.0,24.0,19.0,15.0,7680.0,NaN,33.0,NaN,NaN,58.6
1218,Windsor,2019,310.0,NaN,NaN,310.0,29.0,176.0,62.0,109.0,...,27.0,23.0,19.0,14.0,8262.0,53.8,32.2,NaN,43.0,NaN
1219,Windsor,2019,359.0,2.47,49.14,359.0,29.0,173.0,57.0,121.0,...,27.0,22.0,20.0,16.0,7840.0,55.7,32.2,NaN,43.9,NaN


In [6]:
# CLEAN UP NULL VALUES IN DATA
for county in relevant["COUNTY"].unique():
    # slice dataframe to only have data from a single county
    county_data = relevant[relevant["COUNTY"] == county]
    for year in county_data["DATE"].unique():
        # get current slice of data from that year for that county
        current_group = relevant_groupby.get_group((county, year))
        # get the names of the columns that are missing all their data
        missing_cols = current_group.loc[:, current_group.isna().sum() == current_group.shape[0]].columns
        # get the names of the columns that have some missing data but not all
        other_cols = [col for col in current_group.columns if col not in missing_cols]
        # fill in the missing values in the columns in other_cols with their respective medians
        current_group[other_cols] = current_group[other_cols].fillna(current_group[other_cols].median())
        # save values in current group back into original dataframe ('relevant')
        relevant.update(current_group, overwrite=False)

In [7]:
display(relevant.isna().any())
relevant.head()

COUNTY    False
DATE      False
CDSD       True
EMXP       True
PRCP       True
CLDD       True
DT00       True
DT32       True
DX32       True
DX70       True
DX90       True
EMNT       True
EMXT       True
FZF0       True
FZF1       True
FZF2       True
FZF3       True
FZF4       True
FZF5       True
FZF6       True
FZF7       True
FZF8       True
FZF9       True
HTDD       True
TAVG       True
TMAX       True
TMIN       True
SNOW       True
PSUN       True
dtype: bool

,COUNTY,DATE,CDSD,EMXP,PRCP,CLDD,DT00,DT32,DX32,DX70,...,FZF6,FZF7,FZF8,FZF9,HTDD,TAVG,TMAX,TMIN,SNOW,PSUN
0,Addison,1990,NaN,2.52,54.87,NaN,2.0,136.0,55.0,107.0,...,24.0,24.0,17.0,11.0,7198.0,NaN,54.2,38.5,131.3,NaN
1,Addison,1990,NaN,2.60,48.94,NaN,2.0,136.0,55.0,107.0,...,24.0,24.0,17.0,11.0,7198.0,NaN,54.2,38.5,53.2,NaN
2,Addison,1990,NaN,2.40,46.65,NaN,2.0,136.0,55.0,107.0,...,24.0,24.0,17.0,11.0,7198.0,NaN,54.2,38.5,69.4,NaN
3,Addison,1991,NaN,1.90,37.15,NaN,30.0,161.0,63.0,109.0,...,27.0,20.0,20.0,11.0,NaN,42.9,53.6,32.3,89.4,NaN
4,Addison,1991,NaN,1.90,37.15,NaN,30.0,161.0,63.0,109.0,...,27.0,20.0,20.0,11.0,NaN,42.9,53.6,32.3,26.0,NaN


In [8]:
# CLEAN UP REMAINING NULL VALUES
relevant = relevant.interpolate(method="linear", limit_direction="forward")
relevant = relevant.fillna(relevant.median())    
display(relevant.isna().any())
relevant.head()

COUNTY    False
DATE      False
CDSD      False
EMXP      False
PRCP      False
CLDD      False
DT00      False
DT32      False
DX32      False
DX70      False
DX90      False
EMNT      False
EMXT      False
FZF0      False
FZF1      False
FZF2      False
FZF3      False
FZF4      False
FZF5      False
FZF6      False
FZF7      False
FZF8      False
FZF9      False
HTDD      False
TAVG      False
TMAX      False
TMIN      False
SNOW      False
PSUN      False
dtype: bool

,COUNTY,DATE,CDSD,EMXP,PRCP,CLDD,DT00,DT32,DX32,DX70,...,FZF6,FZF7,FZF8,FZF9,HTDD,TAVG,TMAX,TMIN,SNOW,PSUN
0,Addison,1990,314.0,2.52,54.87,314.0,2.0,136.0,55.0,107.0,...,24.0,24.0,17.0,11.0,7198.000000,47.0,54.2,38.5,131.3,99.72
1,Addison,1990,314.0,2.60,48.94,314.0,2.0,136.0,55.0,107.0,...,24.0,24.0,17.0,11.0,7198.000000,47.0,54.2,38.5,53.2,99.72
2,Addison,1990,314.0,2.40,46.65,314.0,2.0,136.0,55.0,107.0,...,24.0,24.0,17.0,11.0,7198.000000,47.0,54.2,38.5,69.4,99.72
3,Addison,1991,314.0,1.90,37.15,314.0,30.0,161.0,63.0,109.0,...,27.0,20.0,20.0,11.0,7693.666667,42.9,53.6,32.3,89.4,99.72
4,Addison,1991,314.0,1.90,37.15,314.0,30.0,161.0,63.0,109.0,...,27.0,20.0,20.0,11.0,8189.333333,42.9,53.6,32.3,26.0,99.72


In [9]:
# Write cleaned data to file
relevant.to_csv("../../data/cleaned/environment/cleaned environment data.csv", index=False)
print("Data cleaned in {} seconds.".format(time.time() - cleaning_time_start))

Data cleaned in 27.90343976020813 seconds.
